In [1]:
import os
from dotenv import load_dotenv

dotenv_path = '.env'
load_dotenv(dotenv_path)

# Print out the environment variables loaded
for key, value in os.environ.items():
    if 'OPENAI' in key:
        print("OpenAI Key founded")
        #print(f"{key}: {value}")

openai_api_key = os.getenv('OPENAI_API_KEY')

#print(f"OpenAI API Key: {openai_api_key}")

OpenAI Key founded


In [2]:
from langchain_openai import OpenAI
from langchain.prompts.prompt import PromptTemplate
llm = OpenAI()

In [13]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("Data/InformeBC2023.pdf")
pages = loader.load()

Ignoring wrong pointing object 8 0 (offset 0)


In [ ]:
print(type(pages))

In [15]:
len(pages)

5

In [16]:
# Create a list of Document objects
from langchain.schema import Document
documents = [Document(page_content=page.page_content) for page in pages]
#print(documents)

In [25]:
print(f"Your document has {len(documents[1].page_content)} characters")

Your document has 3169 characters


**Split the document in small chunks**

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [28]:
document_chunks = text_splitter.split_documents(documents)

In [29]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 8 chunks.


In [32]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

**Load the embeddings to a vector database**

In [36]:
#!pip install faiss-cpu

In [37]:
from langchain.vectorstores import FAISS

In [38]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

**Create a Retrieval Question & Answering Chain**

In [39]:
from langchain.chains import RetrievalQA

In [40]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

**Now we have a Question & Answering APP**

In [41]:
question = """
De que trata el texto? 
Describir en menos de 100 palabras.
"""

In [42]:
QA_chain.run(question)

/Users/mlandron/.pyenv/versions/myenv444/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'\nEl texto trata sobre el desempeño económico de la República Dominicana en el año 2023 y las proyecciones para el año 2024. Se mencionan diferentes sectores que han tenido un impacto positivo en la economía, como la construcción, las actividades generadoras de divisas y la inversión extranjera. También se destaca el rol del gobierno en la provisión de apoyo técnico y financiero a los productores agropecuarios y la importancia de las remesas enviadas por la diáspora. Se mencionan cifras y estadísticas que reflejan un buen desempeño económico y se proyecta un crecimiento del 5% para el año 2024. '

In [44]:
question2 = """
que significa IMAE?
"""

In [45]:
QA_chain.run(question2)

'\nIMAE significa Indicador Mensual de Actividad Económica. Es un indicador que mide el nivel de actividad económica en un país en un periodo determinado.'

## Connect RAG App with FastAPI

In [47]:
#!pip install fastapi
#!pip install uvicorn

In [ ]:
from fastapi import FastAPI, HTTPException
import uvicorn

app = FastAPI()

@app.post("/conversation")
async def conversation(query: str):
    try:
        result = QA_chain.run(query=query)
        return {"response": result}
    except Exception as e:
        raise HTTPException(detail=str(e), status_code=500)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=5566)

In [ ]:
await server.serve()

* Open http://0.0.0.0:5566/docs#
* In Windows, open http://localhost:5566/docs#

**If you are using VS Code editor, use the following instead and then execute python filename.py in your terminal:**

In [ ]:
# if __name__ == "__main__":
#     import uvicorn

#     uvicorn.run(app, host="0.0.0.0", port=5566)